# Filtering and Downloading Scenes as a GeoPackage

In this tutorial, we will use the catalog download resource to get the GeoPackage schema, then to discover pages of published scenes from which we will populate an in-memory GeoPackage collection using [fiona](https://github.com/Toblerity/Fiona), and finally we will download the populated GeoPackage. See the [GeoPackage download API documentation](https://api.carbonmapper.org/api/v1/docs#/Data%20Catalog/catalog_api_scene_scenes_download_gpkg).

The resource provides the `intersects` parameter, which accepts a feature's GeoJSON geometry and will filter for scenes that intersect the provided geometry. Our GeoJSON is a simplified representation of the state of California. Note that there is an 8,192 byte API URL size limit for requests, hence the simplified GeoJSON. We will also provide a 3-year date range for our query. Because the endpoint is paginated, the `count` returned may differ from the length of the `items` returned.

## Installing Requirements

For tutorial purposes, we will use IPython's [pip](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-pip) magic command to install the [Fiona](https://github.com/Toblerity/Fiona) and [Requests](https://pypi.org/project/requests/) requirements in the current kernel.

In [ ]:
%pip install fiona requests

## Requesting an Access Token

The catalog download endpoint requires a platform API token. We will use the platform API to obtain the token. Obtaining a token requires a platform account. Information about registering for an account and manually obtaining a token using the platform website can be found in the [API Authentication](https://github.com/carbon-mapper/platform-public/tree/main/tutorials#api-authentication) section of the tutorials README.

In [ ]:
import getpass

import requests

base_url = "https://api.carbonmapper.org/api/v1/"

# Request an account token
account_token_path = "token/pair"
account_email = input("Enter the email address associated with your Carbon Mapper account: ")
account_password = getpass.getpass("Enter the password associated with your Carbon Mapper account: ")
payload = {
    "email": account_email,
    "password": account_password,
}

response = requests.post(
    f"{base_url}{account_token_path}",
    json=payload,
)
response.raise_for_status()

access_token = response.json()["access"]
print(f"Access token: {access_token}")

## Filtering and Downloading Scenes

In [ ]:
import os
import pathlib

from fiona import Feature
from fiona.io import MemoryFile

geopackage_path = "catalog/download/scenes.gpkg"
payload = {
    "intersects": '{"type": "MultiPolygon", "coordinates": [[[[-120.248484, 33.999329], [-120.043259, 34.035806], [-119.97026, 33.944359], [-120.121817, 33.895712], [-120.248484, 33.999329]]], [[[-119.789798, 34.05726], [-119.637742, 34.013178], [-119.52064, 34.034262], [-119.758141, 33.959212], [-119.923337, 34.069361], [-119.789798, 34.05726]]], [[[-120.46258, 34.042627], [-120.368278, 34.076465], [-120.302122, 34.023574], [-120.35793, 34.015029], [-120.46258, 34.042627]]], [[[-119.543842, 33.280329], [-119.465717, 33.259239], [-119.429559, 33.228167], [-119.545872, 33.233406], [-119.543842, 33.280329]]], [[[-119.422972, 34.004368], [-119.441226, 34.014075], [-119.357462, 34.015919], [-119.391551, 34.002505], [-119.422972, 34.004368]]], [[[-118.524531, 32.895488], [-118.605534, 33.030999], [-118.353504, 32.821962], [-118.425634, 32.800595], [-118.524531, 32.895488]]], [[[-118.500212, 33.449592], [-118.370323, 33.409285], [-118.305084, 33.310323], [-118.60403, 33.47654], [-118.500212, 33.449592]]], [[[-122.418698, 37.852717], [-122.434403, 37.852434], [-122.446316, 37.861046], [-122.421341, 37.869946], [-122.418698, 37.852717]]], [[[-123.013916, 37.700355], [-123.004489, 37.706262], [-122.997189, 37.697909], [-123.005543, 37.689392], [-123.013916, 37.700355]]], [[[-122.3785, 37.826505], [-122.369941, 37.832137], [-122.362661, 37.807577], [-122.372422, 37.811301], [-122.3785, 37.826505]]], [[[-124.065521, 41.464739], [-124.211605, 41.99846], [-119.999168, 41.99454], [-120.001014, 38.999574], [-114.633013, 35.002085], [-114.131489, 34.260387], [-114.43338, 34.088413], [-114.535664, 33.568788], [-114.721233, 33.396912], [-114.496284, 32.822326], [-117.124862, 32.534156], [-117.469794, 33.296417], [-118.132698, 33.753217], [-118.411211, 33.741985], [-118.519514, 34.027509], [-119.130169, 34.100102], [-119.559459, 34.413395], [-120.637805, 34.56622], [-120.644311, 35.139616], [-121.888491, 36.30281], [-121.978592, 36.580488], [-121.814462, 36.682858], [-121.862266, 36.931552], [-122.405073, 37.195791], [-122.514483, 37.780829], [-122.111344, 37.50758], [-122.430087, 37.963115], [-122.273006, 38.07438], [-122.489974, 38.112014], [-122.505383, 37.822128], [-123.024066, 37.994878], [-122.977082, 38.267902], [-123.725367, 38.917438], [-123.851714, 39.832041], [-124.409591, 40.438076], [-124.137066, 40.925732], [-124.065521, 41.464739]]]]}',
    "datetime": "2020-01-01/2023-12-31",
    "offset": 0,
}
# Full path at which to store the file
# If empty, will save to current working directory
file_path = pathlib.Path("")
file_name = "scenes.gpkg"
abs_path = os.path.abspath(file_path / file_name)
headers = {"Authorization": f"Bearer {access_token}"}
print("Downloading features.")

# Request a GeoPackage with 0 features
response = requests.get(
    f"{base_url}{geopackage_path}",
    params={"limit": 0},
    headers=headers,
)
response.raise_for_status()

# Get the schema from the empty GeoPackage
# The filename argument is not required, but is included to suppress a confusing warning
with MemoryFile(response.content, filename="schema_file.gpkg") as schema_file:
    with schema_file.open() as schema_collection:
        schema = schema_collection.schema

with MemoryFile() as dst_file:
    # Instantiate an  in-memory GeoPackage using the previously retrieved schema
    with dst_file.open(mode="w", driver="GPKG", schema=schema) as dst_collection:
        while True:
            # Download a page of features
            response = requests.get(
                f"{base_url}{geopackage_path}",
                params=payload,
                headers=headers,
            )
            response.raise_for_status()

            count = int(response.headers["pagination-count"])
            limit = int(response.headers["pagination-limit"])
            offset = int(response.headers["pagination-offset"])

            # Load the source GeoPackage into an in-memory file
            with MemoryFile(response.content) as src_file:
                with src_file.open() as src_collection:
                    # Add the source features into the destination collection
                    for feature in src_collection:
                        dst_collection.write(Feature(geometry=feature.geometry, properties=feature.properties))

                    print(f"Added {len(list(src_collection))} features to scenes GeoPackage.") 


            payload["offset"] = int(offset + limit)

            # No more records to download
            if payload["offset"] >= count:
                break

    confirm = input("{action} file ({abs_path})? Y/n".format(
        action="Overwrite" if pathlib.Path(abs_path).is_file() else "Save",
        abs_path=abs_path,
    ))

    if confirm.lower() == "y":
        # Save the GeoPackage to disk
        with open(abs_path, "wb") as file:
            file.write(dst_file.getbuffer())

        print(f"Scenes downloaded to {abs_path}.")
